In [ ]:
for name in list(globals().keys()):
    if name[0] != "_":  # Avoid deleting internal Python variables
        del globals()[name]
import requests, zipfile, re, os, shutil, cftime, ravenpy
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from siphon.catalog import TDSCatalog
from netCDF4 import Dataset
## Extracting example models
url = "https://github.com/rarabzad/Pavics_tools/raw/refs/heads/main/SMM_grids_hrus.zip"
with open("SMM_grids_hrus.zip", "wb") as f:
    f.write(requests.get(url).content)
with zipfile.ZipFile("SMM_grids_hrus.zip", "r") as zip_ref:
    zip_ref.extractall()
os.remove("SMM_grids_hrus.zip")
url = "https://github.com/rarabzad/Pavics_tools/raw/refs/heads/main/SMM_models.zip"
with open("SMM_models.zip", "wb") as f:
    f.write(requests.get(url).content)
with zipfile.ZipFile("SMM_models.zip", "r") as zip_ref:
    zip_ref.extractall()
os.remove("SMM_models.zip")
## Copy CC forcing grid weights file
shutil.copy('SMM_grids_hrus/GridWeights.txt', '/notebook_dir/writable-workspace/SMM_models/milk/input/')
shutil.copy('SMM_grids_hrus/GridWeights.txt', '/notebook_dir/writable-workspace/SMM_models/stmary/input/')
## downloading and processing climate change data
hrufile_path = "SMM_grids_hrus/hru.shp"
# Load the main function
exec(requests.get("https://raw.githubusercontent.com/rarabzad/Pavics_tools/refs/heads/main/ESPO_G6_R2_Downloader.py").text)
# URL for the TDS catalog
url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/catalog/datasets/simulations/bias_adjusted/cmip6/ouranos/ESPO-G/ESPO-G6-R2v1.0.0/catalog.xml"
# Access the catalog
cat = TDSCatalog(url)
# Retrieve the datasets directly (these will be strings, not objects)
datasets = [dataset for dataset in cat.datasets]  # cat.datasets contains strings, no need for .name
# function for changing the forcing functions file name
def replace_filename(file_path, old, new):
    with open(file_path, 'r+') as file:
        content = file.read().replace(old, new)
        file.seek(0)
        file.write(content)
# Apply the ESPO_G6_R2_Downloader function over all climate models and scenarios
old_forcing = 'RavenInput.nc'
old_start_date = 'start_date'
old_end_date = 'end_date'
os.mkdir('hydrographs')
os.mkdir('climateForcings')
for file in datasets:
    start_index = file.find("ScenarioMIP_NAM_") + len("ScenarioMIP_NAM_")
    end_index = file.find("_ssp")
    model_name = file[start_index:end_index]
    scenario = file[end_index + 1:end_index + 7]
    ESPO_G6_R2_Downloader(hrufile_path, model_name, scenario)
    new_forcing = f"Raven_input_{model_name}_{scenario}.nc"
    replace_filename('/notebook_dir/writable-workspace/SMM_models/milk/milk.rvt'    , old_forcing, new_forcing)
    replace_filename('/notebook_dir/writable-workspace/SMM_models/stmary/stmary.rvt', old_forcing, new_forcing)
    shutil.copy(new_forcing, os.path.join('/notebook_dir/writable-workspace/SMM_models/stmary/input/', new_forcing))
    shutil.copy(new_forcing, os.path.join('/notebook_dir/writable-workspace/SMM_models/milk/input/', new_forcing))
    nc_file = Dataset(new_forcing, 'r')
    calendar_type = nc_file['time'].getncattr('calendar')
    replace_filename('/notebook_dir/writable-workspace/SMM_models/milk/milk.rvt'    , old_forcing, new_forcing)
    replace_filename('/notebook_dir/writable-workspace/SMM_models/stmary/stmary.rvt', old_forcing, new_forcing)
    if calendar_type == 'noleap':
        shutil.copy(new_forcing, os.path.join('/notebook_dir/writable-workspace/SMM_models/stmary/input/', new_forcing))
        shutil.copy(new_forcing, os.path.join('/notebook_dir/writable-workspace/SMM_models/milk/input/', new_forcing))
        times = cftime.num2date(nc_file.variables['time'][:], units=nc_file.variables['time'].units)
        new_start_date = times[1].strftime('%Y-%m-%d')  + " 00:00:00"
        new_end_date   = times[-2].strftime('%Y-%m-%d') + " 00:00:00"
        nc_file.close()
        replace_filename('/notebook_dir/writable-workspace/SMM_models/milk/milk.rvi'      , old_start_date, new_start_date)
        replace_filename('/notebook_dir/writable-workspace/SMM_models/stmary/stmary.rvi'  , old_start_date, new_start_date)
        replace_filename('/notebook_dir/writable-workspace/SMM_models/milk/milk.rvi'      , old_end_date,   new_end_date)
        replace_filename('/notebook_dir/writable-workspace/SMM_models/stmary/stmary.rvi'  , old_end_date,   new_end_date)
        ravenpy.run(modelname = 'milk' , configdir = '/notebook_dir/writable-workspace/SMM_models/milk/')
        ravenpy.run(modelname = 'stmary' , configdir = '/notebook_dir/writable-workspace/SMM_models/stmary/')
        source_file = '/notebook_dir/writable-workspace/SMM_models/milk/output/Hydrographs.csv'
        shutil.copy(source_file, os.path.join('hydrographs',f"milk_{os.path.splitext(new_forcing.replace('Raven_input_', ''))[0]}.csv"))
        source_file = '/notebook_dir/writable-workspace/SMM_models/stmary/output/Hydrographs.csv'
        shutil.copy(source_file, os.path.join('hydrographs',f"stmary_{os.path.splitext(new_forcing.replace('Raven_input_', ''))[0]}.csv"))
        os.remove(os.path.join('/notebook_dir/writable-workspace/SMM_models/stmary/input/', new_forcing))
        os.remove(os.path.join('/notebook_dir/writable-workspace/SMM_models/milk/input/', new_forcing))
        old_forcing    = new_forcing
        old_start_date = new_start_date
        old_end_date   = new_end_date
        shutil.move(new_forcing, 'climateForcings')
    else:
        shutil.move(new_forcing, 'climateForcings')
        old_forcing    = new_forcing
        old_start_date = new_start_date
        old_end_date   = new_end_date
        print(f"Skipping iteration {new_forcing} because calendar is not 'noleap'. Current calendar: {calendar_type}")
shutil.make_archive('hydrographs.zip', 'zip', 'hydrographs')